In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# In-Memory Assembly of Jacobians

When you have groups, or entire models, that are small enough to fit the entire Jacobian into memory, you can have OpenMDAO actually assemble the partial-derivative Jacobian in memory. In many cases this can yield a substantial speed up over the default, [matrix-free](theory-assembled-vs-matrix-free) implementation in OpenMDAO.

```{Note}
Assembled Jacobians are especially effective when you have a deeply-nested hierarchy with a large number of components and/or variables. See the [matrix-free](theory-assembled-vs-matrix-free) for more details on how to best select which type of Jacobian to use.

```

To use an assembled Jacobian, you set the `assemble_jac` option of the linear solver that will use it to True. The type of the assembled Jacobian will be determined by the value of `options['assembled_jac_type']` in the solver’s containing system. There are two options of `assembled_jac_type` to choose from, `dense` and `csc`.



```{Note}
`csc` is an abbreviation for compressed sparse column. `csc` is one of many sparse storage schemes that allocate contiguous storage in memory for the nonzero elements of the matrix, and perhaps a limited number of zeros. For more information, see [Compressed sparse column](https://en.wikipedia.org/wiki/Sparse_matrix).
```

For example:

    model.options['assembled_jac_type'] = 'dense'
    model.linear_solver = DirectSolver(assemble_jac=True)

‘csc’ is the default, and you should try that first if you’re not sure of which one to use. Most problems, even if they have dense sub-Jacobians from each component, are fairly sparse at the model level and the [DirectSolver](../../building_blocks/solvers/direct_solver) will usually be much faster with a sparse factorization.

```{Note}
You are allowed to use multiple assembled Jacobians at multiple different levels of your model hierarchy. This may be useful if you have nested nonlinear solvers to converge very difficult problems.
```

## Usage Example

In the following example, borrowed from the [newton solver tutorial](../../../advanced_user_guide/models_implicit_components/models_with_solvers_implicit), we assemble the Jacobian at the same level of the model hierarchy as the [NewtonSolver](../../building_blocks/solvers/newton)  and [DirectSolver](../../building_blocks/solvers/direct_solver). In general, you will always do the assembly at the same level as the linear solver that will make use of the Jacobian matrix.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src56", get_code("openmdao.test_suite.scripts.circuit_analysis.Resistor"), display=False)

:::{Admonition} `Resistor` class definition 
:class: dropdown

{glue:}`code_src56`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src57", get_code("openmdao.test_suite.scripts.circuit_analysis.Diode"), display=False)

:::{Admonition} `Diode` class definition 
:class: dropdown

{glue:}`code_src57`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src58", get_code("openmdao.test_suite.scripts.circuit_analysis.Node"), display=False)

:::{Admonition} `Node` class definition 
:class: dropdown

{glue:}`code_src58`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.scripts.circuit_analysis import Resistor, Diode, Node

class Circuit(om.Group):

    def setup(self):
        self.add_subsystem('n1', Node(n_in=1, n_out=2), promotes_inputs=[('I_in:0', 'I_in')])
        self.add_subsystem('n2', Node())  # leaving defaults

        self.add_subsystem('R1', Resistor(R=100.), promotes_inputs=[('V_out', 'Vg')])
        self.add_subsystem('R2', Resistor(R=10000.))
        self.add_subsystem('D1', Diode(), promotes_inputs=[('V_out', 'Vg')])

        self.connect('n1.V', ['R1.V_in', 'R2.V_in'])
        self.connect('R1.I', 'n1.I_out:0')
        self.connect('R2.I', 'n1.I_out:1')

        self.connect('n2.V', ['R2.V_out', 'D1.V_in'])
        self.connect('R2.I', 'n2.I_in:0')
        self.connect('D1.I', 'n2.I_out:0')

        self.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
        self.nonlinear_solver.options['iprint'] = 2
        self.nonlinear_solver.options['maxiter'] = 20
        ##################################################################
        # Assemble at the group level. Default assembled jac type is 'csc'
        ##################################################################
        self.options['assembled_jac_type'] = 'csc'
        self.linear_solver = om.DirectSolver(assemble_jac=True)

p = om.Problem()
model = p.model

model.add_subsystem('circuit', Circuit())

p.setup()

p.set_val('circuit.I_in', 0.1)
p.set_val('circuit.Vg', 0.)

# set some initial guesses
p.set_val('circuit.n1.V', 10.)
p.set_val('circuit.n2.V', 1.)

p.run_model()

In [ ]:
print(p['circuit.n1.V'])
print(p['circuit.n2.V'])
print(p['circuit.R1.I'])
print(p['circuit.R2.I'])
print(p['circuit.D1.I'])

# sanity check: should sum to .1 Amps
print(p['circuit.R1.I'] + p['circuit.D1.I'])

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p['circuit.n1.V'], 9.90804735, tolerance=1e-5)
assert_near_equal(p['circuit.n2.V'], 0.71278185, tolerance=1e-5)
assert_near_equal(p['circuit.R1.I'], 0.09908047, tolerance=1e-5)
assert_near_equal(p['circuit.R2.I'], 0.00091953, tolerance=1e-5)
assert_near_equal(p['circuit.D1.I'], 0.00091953, tolerance=1e-5)